In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

In [ ]:
df = pd.read_csv('base_clientes.csv')
df.head()

,id_cliente,persona,engajamento,compras_60_dias,proba
0,cli_0000,esporadico,87.1,47.0,0.34
1,cli_0001,unknown,7.9,0.0,0.07
2,cli_0002,esporadico,85.3,116.0,0.34
3,cli_0003,esporadico,9.0,159.0,0.14
4,cli_0004,esporadico,2.5,0.0,0.11


In [ ]:
df.describe()

,engajamento,compras_60_dias,proba
count,10000.000000,10000.000000,10000.000000
mean,27.472650,39.009100,0.163968
std,30.801517,60.223568,0.097923
min,0.000000,0.000000,0.050000
25%,4.400000,0.000000,0.090000
50%,11.600000,0.000000,0.120000
75%,50.825000,89.000000,0.240000
max,99.600000,226.000000,0.490000


In [ ]:
df.groupby('persona')['id_cliente'].nunique()

persona
churrasco_fds          967
esporadico            3044
unknown               4456
vinho_final_do_dia    1533
Name: id_cliente, dtype: int64

In [ ]:
#control
df_controle = df.sample(500, random_state = 123)

#train
lista_controle = list(df_controle.index)
df_tratamento = df[~df.index.isin(lista_controle)].sample(500, random_state = 123)

In [ ]:
#IDs verification
lista_final = []

for item in df_controle.index:
  lista_final.append(item)

for item in df_tratamento.index:
  lista_final.append(item)

len(set(lista_final))


1000

In [ ]:
#control base

df_controle.describe()

,engajamento,compras_60_dias,proba
count,500.000000,500.000000,500.000000
mean,25.539200,38.488000,0.154220
std,30.465644,59.973824,0.095113
min,0.000000,0.000000,0.050000
25%,4.000000,0.000000,0.080000
50%,10.100000,0.000000,0.120000
75%,37.325000,89.000000,0.230000
max,96.200000,200.000000,0.460000


In [ ]:
#train base
df_tratamento.describe()

,engajamento,compras_60_dias,proba
count,500.000000,500.000000,500.000000
mean,27.661200,42.270000,0.163940
std,30.609359,61.708906,0.099489
min,0.000000,0.000000,0.050000
25%,4.500000,0.000000,0.090000
50%,11.650000,0.000000,0.120000
75%,54.050000,90.500000,0.240000
max,96.200000,199.000000,0.470000


In [ ]:
from scipy.stats import ttest_ind

if ttest_ind(df_controle['engajamento'], df_tratamento['engajamento'])[1] >0.05:
  print("Variável de engajamento tem média estatisticamente igual nos grupos de controle e teste")
else:
  print("Variável de engajamento não tem média estatisticamente igual nos grupos de controle e teste")

if ttest_ind(df_controle['compras_60_dias'], df_tratamento['compras_60_dias'])[1] >0.05:
  print("Variável de compras_60_dias tem média estatisticamente igual nos grupos de controle e teste")
else:
  print("Variável de compras_60_dias não tem média estatisticamente igual nos grupos de controle e teste")


if ttest_ind(df_controle['proba'], df_tratamento['proba'])[1] >0.05:
  print("Variável de proba tem média estatisticamente igual nos grupos de controle e teste")
else:
  print("Variável de proba não tem média estatisticamente igual nos grupos de controle e teste")


Variável de engajamento tem média estatisticamente igual nos grupos de controle e teste
Variável de compras_60_dias tem média estatisticamente igual nos grupos de controle e teste
Variável de proba tem média estatisticamente igual nos grupos de controle e teste


In [ ]:
#API Connection
import requests
from datetime import datetime
import pandas as pd
import time
import json

urlbase = 'https://bweb-server.herokuapp.com/'
def get_result(x):
    try:
        result = pd.DataFrame.from_dict(x.json())
    except:
        result = x.text
    return result

In [ ]:
## A/B Test on the API
# checking budget and token

url = urlbase + 'check_wallet'
payload = {'token': 'b18651a1b274944919efe134a30e37068216c2c1fd2790666dbd5bf7b069fdce'}
x = requests.post(url, data = payload)
res1 = get_result(x)
print(res1)

{"grupo":"Grupo 4","money_current":0.0,"money_initial":5500.0}



In [ ]:
#control n train segregation
clientes_controle = list(df_controle['id_cliente'])
clientes_tratamento = list(df_tratamento['id_cliente'])

# API:
clientes_controle = ','.join(clientes_controle)
clientes_tratamento = ','.join(clientes_tratamento)

# API
payload = {'token': 'b18651a1b274944919efe134a30e37068216c2c1fd2790666dbd5bf7b069fdce',
         'controle': clientes_controle,
          'tratamento': clientes_tratamento}


url = urlbase + 'experiment'
x = requests.post(url, data = payload)
# A função get_result já retorna no format de pandas DataFrame
res = get_result(x)
res

,grupo,id_cliente,vendas
0,controle,cli_0007,89.0
1,controle,cli_0014,0.0
2,controle,cli_0019,199.0
3,controle,cli_0156,0.0
4,controle,cli_0207,119.0
...,...,...,...
995,tratamento,cli_9897,159.0
996,tratamento,cli_9923,96.0
997,tratamento,cli_9929,0.0
998,tratamento,cli_9946,41.0


In [ ]:
df_results = df.merge(res, on = 'id_cliente', how = 'inner')
df_results

,id_cliente,persona,engajamento,compras_60_dias,proba,grupo,vendas
0,cli_0006,esporadico,59.7,129.0,0.27,tratamento,0.0
1,cli_0007,unknown,25.4,0.0,0.11,controle,89.0
2,cli_0014,esporadico,88.5,159.0,0.35,controle,0.0
3,cli_0016,unknown,2.9,0.0,0.06,tratamento,0.0
4,cli_0019,vinho_final_do_dia,16.1,99.0,0.14,controle,199.0
...,...,...,...,...,...,...,...
995,cli_9923,unknown,2.9,0.0,0.06,tratamento,96.0
996,cli_9929,churrasco_fds,13.2,0.0,0.26,tratamento,0.0
997,cli_9944,esporadico,2.4,0.0,0.11,controle,0.0
998,cli_9946,churrasco_fds,3.4,0.0,0.23,tratamento,41.0


In [ ]:
#from google.colab import files

#df_results.to_csv('output.csv', encoding = 'utf-8-sig') 
#files.download('output.csv')

In [ ]:
df_results = pd.read_csv('output.csv')
df_results.head()

,Unnamed: 0,id_cliente,persona,engajamento,compras_60_dias,proba,grupo,vendas
0,0,cli_0006,esporadico,59.7,129.0,0.27,tratamento,0.0
1,1,cli_0007,unknown,25.4,0.0,0.11,controle,89.0
2,2,cli_0014,esporadico,88.5,159.0,0.35,controle,0.0
3,3,cli_0016,unknown,2.9,0.0,0.06,tratamento,0.0
4,4,cli_0019,vinho_final_do_dia,16.1,99.0,0.14,controle,199.0


In [ ]:
df_results = df_results.iloc[:,1:8]

In [ ]:
df_results.groupby('grupo').mean()

,engajamento,compras_60_dias,proba,vendas
grupo,,,,
controle,25.5392,38.488,0.15422,37.016
tratamento,27.6612,42.270,0.16394,37.008


In [ ]:
def check_vendas(x):
  if x == 0:
    return "Nao"
  else:
    return "Sim"

df_results2 = df_results
df_results2['houve_venda'] = df_results2['vendas'].apply(lambda x: check_vendas(x))

df_results2.groupby(['grupo', 'houve_venda']).count()

id_cliente  persona  engajamento  compras_60_dias  \
grupo      houve_venda                                                      
controle   Nao                 337      337          337              337   
           Sim                 163      163          163              163   
tratamento Nao                 299      299          299              299   
           Sim                 201      201          201              201   

                        proba  vendas  
grupo      houve_venda                 
controle   Nao            337     337  
           Sim            163     163  
tratamento Nao            299     299  
           Sim            201     201

In [ ]:
# group tax convertion

print("Taxa de Conversão - Grupo Controle: ",(163/500)*100)
print("Taxa de Conversão - Grupo Tratamento: ",(201/500)*100)

Taxa de Conversão - Grupo Controle:  32.6
Taxa de Conversão - Grupo Tratamento:  40.2


In [ ]:
df_results.groupby(['grupo', 'persona']).mean()

engajamento  compras_60_dias     proba  \
grupo      persona                                                      
controle   churrasco_fds         23.448718        48.897436  0.288205   
           esporadico            26.783553        38.809211  0.178026   
           unknown               25.723077        36.417004  0.117611   
           vinho_final_do_dia    23.070968        39.403226  0.157419   
tratamento churrasco_fds         37.611628        58.906977  0.326279   
           esporadico            24.777419        32.690323  0.172258   
           unknown               27.027064        44.440367  0.121743   
           vinho_final_do_dia    29.534524        45.797619  0.175000   

                                  vendas  
grupo      persona                        
controle   churrasco_fds       46.666667  
           esporadico          42.519737  
           unknown             33.016194  
           vinho_final_do_dia  33.387097  
tratamento churrasco_fds       47.906977  
           esporadico          39.690323  
           unknown             34.307339  
           vinho_final_do_dia  33.488095

In [ ]:
# T-Teste Student

ttest_ind(df_results[df_results['grupo']=='tratamento']['vendas'], df_results[df_results['grupo']=='controle']['vendas'])

Ttest_indResult(statistic=-0.0021805032284407592, pvalue=0.9982606472833955)

In [ ]:
from sklearn import metrics

In [ ]:
def check_vendas_n(x):
  if x <= 11:
    return 0
  else:
    return 1

df_results2['venda_n'] = df_results2['vendas'].apply(lambda x: check_vendas_n(x))

In [ ]:
df_trat = df_results2.loc[df_results2['grupo'] == 'tratamento']

In [ ]:
y = df_trat['venda_n'].to_numpy()

In [ ]:
ypred = df_trat['proba'].to_numpy()

In [ ]:
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score

In [ ]:
#M_vendas =df_trat.loc[df_results2['vendas'] != 0].iloc[:,6].mean()
M_vendas =df_trat.iloc[:,6].mean()

M_vendas

37.008

In [ ]:
#df_trat.loc[df_trat['proba'] >= 0.1].count()
#df_trat.loc[df_trat['venda_n'] == 1].count()
df_trat.loc[df_trat['proba'] >= 0.1].loc[df_trat['venda_n'] == 1].count()

id_cliente         147
persona            147
engajamento        147
compras_60_dias    147
proba              147
grupo              147
vendas             147
houve_venda        147
venda_n            147
dtype: int64

In [ ]:
VTP = - 11 + M_vendas
VFP = - 11
df = pd.DataFrame()
#array_thresh = [0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0, 1.1]
array_thresh = [0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55, 0.6]
value_total_vec = [None] * len(array_thresh)

for i in range(0,len(array_thresh)):
    threshold = array_thresh[i]
    y_pred_round = [1 if x >= threshold else 0 for x in ypred]
    CM = confusion_matrix(y, y_pred_round)
    TN = CM[0][0] # 0 -> nao fiz envio, mas não efetuei a venda
    FN = CM[1][0] # 0 -> nao fiz envio porem vendi
    TP = CM[1][1] # +++ -> Fiz o envio, efetuei a venda
    FP = CM[0][1] # --- -> Fiz o envio, mas não efetuei a venda
    valor = (TN * 0) + (FN * 0) + (TP * VTP) + (FP * VFP)
    value_total_vec[i] = valor
    df = df.append(pd.DataFrame.from_dict({'threshold': [threshold],'TN': [TN],'FN': [FN],'TP': [TP],'FP': [FP], 'value_total_vec': [valor]}))

df

,threshold,TN,FN,TP,FP,value_total_vec
0,0.00,0,0,195,305,1716.560
0,0.05,0,0,195,305,1716.560
0,0.10,86,48,147,219,1414.176
0,0.15,199,103,92,106,1226.736
0,0.20,215,122,73,90,908.584
0,0.25,245,135,60,60,900.480
0,0.30,272,163,32,33,469.256
0,0.35,289,183,12,16,136.096
0,0.40,296,189,6,9,57.048
0,0.45,305,192,3,0,78.024


In [ ]:
import plotly.express as px

px.line(df, x = 'threshold', y = 'value_total_vec')

In [ ]:
# Exportar em PDF

!wget -nc https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
from colab_pdf import colab_pdf
colab_pdf('bweb-client-example.ipynb')

--2022-05-30 01:49:38--  https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1864 (1.8K) [text/plain]
Saving to: ‘colab_pdf.py’

colab_pdf.py        100%[===================>]   1.82K  --.-KB/s    in 0s      

2022-05-30 01:49:39 (25.9 MB/s) - ‘colab_pdf.py’ saved [1864/1864]

Mounted at /content/drive/




Extracting templates from packages: 100%
[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab Notebooks/bweb-client-example.ipynb to pdf
/usr/local/lib/python3.7/dist-packages/nbconvert/filters/datatypefilter.py:41: UserWarning: Your element with mimetype(s) dict_keys(['text/html']) is not able to be represented.
  mimetypes=output.keys())
[NbConvertApp] Writing 63416 bytes to ./n

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'File ready to be Downloaded and Saved to Drive'